In [146]:
from numpy import zeros
from numpy import ones
from numpy import expand_dims
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Concatenate
from keras.initializers import RandomNormal
from matplotlib import pyplot
import keras as K
import glob
import os

In [147]:
import tensorflow as tf
import pandas as pd
import numpy as np
import glob
import os
from keras.utils import to_categorical
from sklearn.metrics import classification_report

In [148]:
from sklearn.preprocessing import MinMaxScaler  # 0~1
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler

In [149]:
df2 = pd.read_csv('F:/HAR/WISDM/realdata.csv', index_col=0)

In [150]:
df2

,user,activity,time,x,y,z
0,33,Jogging,49105962326000,-0.694638,12.680544,0.503953
1,33,Jogging,49106062271000,5.012288,11.264028,0.953424
2,33,Jogging,49106112167000,4.903325,10.882658,-0.081722
3,33,Jogging,49106222305000,-0.612916,18.496431,3.023717
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164
...,...,...,...,...,...,...
343411,11,Walking,1786872234000,0.310000,10.730000,0.040861
343412,11,Walking,1786922282000,2.760000,10.950000,0.844462
343413,11,Walking,1786972392000,0.800000,10.150000,1.334794
343414,11,Walking,1787022227000,-1.570000,8.690000,1.416516


In [151]:
df2 = df2[['x', 'y', 'z', 'activity']]

In [152]:
df2 

,x,y,z,activity
0,-0.694638,12.680544,0.503953,Jogging
1,5.012288,11.264028,0.953424,Jogging
2,4.903325,10.882658,-0.081722,Jogging
3,-0.612916,18.496431,3.023717,Jogging
4,-1.184970,12.108489,7.205164,Jogging
...,...,...,...,...
343411,0.310000,10.730000,0.040861,Walking
343412,2.760000,10.950000,0.844462,Walking
343413,0.800000,10.150000,1.334794,Walking
343414,-1.570000,8.690000,1.416516,Walking


In [153]:
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
df2['activity'] = label.fit_transform(df2['activity'])
df2.head()

<ipython-input-153-94d60b1f1642>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['activity'] = label.fit_transform(df2['activity'])


,x,y,z,activity
0,-0.694638,12.680544,0.503953,1
1,5.012288,11.264028,0.953424,1
2,4.903325,10.882658,-0.081722,1
3,-0.612916,18.496431,3.023717,1
4,-1.184970,12.108489,7.205164,1


In [154]:
df2['activity'].value_counts()

activity
5    137375
1    129392
4     35137
0     33358
2      4599
3      3555
Name: count, dtype: int64

In [155]:
err_train = df2.values[:, :-1]
status_train = df2.values[:, -1].astype(np.int64)
status_train_cat = to_categorical(status_train)

In [156]:
err_train.shape

(343416, 3)

In [157]:
status_train_cat.shape

(343416, 6)

In [158]:
df2.shape

(343416, 4)

In [159]:
status_train.shape

(343416,)

In [160]:
status_train_cat.shape

(343416, 6)

In [161]:
err_train.shape

(343416, 3)

In [162]:
len(err_train[0])

3

In [179]:
scaler = MinMaxScaler()
err_train = scaler.fit_transform(err_train)

In [180]:
def define_discriminator(in_shape=(10,1), n_classes=6):
    # weight initialization
    inputs = tf.keras.Input(shape=(len(err_train[0]),))
    fe = Dense(16)(inputs)
    fe = tf.nn.relu(fe)
    fe = Dense(16)(fe)
    fe = tf.nn.relu(fe)
    # real/fake output
    out1 = Dense(1, activation='sigmoid')(fe)
    # class label output
    out2 = Dense(n_classes, activation='softmax')(fe)
    # define model
    model = Model(inputs, [out1, out2])
    # compile model
    opt = Adam(lr=0.0001, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
    return model

In [181]:
def define_generator(latent_dim, n_classes=6):
    in_lat = Input(shape=(latent_dim,))
    label = Input(shape=(1,), dtype='int32')
    
    n_nodes = 16
    gen = Dense(n_nodes)(in_lat)
    gen = tf.nn.tanh(gen)
    gen = Dense(16)(gen)
    gen = tf.nn.tanh(gen)
    out_layer = Dense(len(err_train[0]), activation='linear')(gen)
    # define model
    model = Model([in_lat,label], out_layer)
    return model

In [182]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    
    # connect the outputs of the generator to the inputs of the discriminator
    gan_output = d_model(g_model.output)
    
    # define gan model as taking noise and label and outputting real/fake and label outputs
    model = Model(g_model.input, gan_output)
    
    # compile model
    opt = Adam(lr=0.0001, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
    return model

In [183]:
# load images
def load_real_samples():
    (trainX, trainy) = (err_train, status_train)
    print(trainX.shape, trainy.shape)
    return [trainX, trainy]

In [184]:
# select real samples
def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return [X, labels], y

In [185]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes = 6):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = randint(0, n_classes-1, n_samples)
    return [z_input, labels]


In [186]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict([z_input, labels_input])
    # create class labels
    y = zeros((n_samples, 1))
    return [images, labels_input], y

In [187]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, d_model, latent_dim, n_samples=100):

    X, y = dataset
    y_pred = d_model.predict(X)
    y_pred = y_pred[1:]
    y_pred = np.argmax(y_pred, axis=2)

    print(classification_report(y, y_pred[0,:]))

    filename1 = 'C:/Users/GC/gmodel/generated_plot_%04d.png' % (step + 1)
    pyplot.savefig(filename1)
    pyplot.close()
    # save the generator model
    filename2 = 'C:/Users/GC/g_model_%04d.h5' % (step + 1)
    g_model.save(filename2)
    filename3 = 'C:/Users/GC/d_model_%04d.h5' % (step + 1)
    d_model.save(filename3)
    filename4 = 'C:/Users/GC/gan_model_%04d.h5' % (step + 1)
    gan_model.save(filename4)
    print('>Saved: %s, %s, %s and %s' % (filename1, filename2, filename3, filename4))

In [188]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=1000):

    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    
    # calculate the size of half a batch of samples
    #half_batch = int(n_batch / 2)
    half_batch = n_batch
    # manually enumerate epochs
    
    for i in range(n_steps):
        # get randomly selected 'real' samples
        [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
        
        # update discriminator model weights
        _, d_r1, d_r2 = d_model.train_on_batch(X_real, [y_real, labels_real])
        
        # generate 'fake' examples
        [X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)

        # print('GENERATE_FAKE_SAMPLES')
        # print(X_fake)
        
        # update discriminator model weights
        _, d_f, d_f2 = d_model.train_on_batch(X_fake, [y_fake, labels_fake])
        
        # prepare points in latent space as input for the generator        
        [z_input, z_labels] = generate_latent_points(latent_dim, n_batch)
        
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 1))
        
        # update the generator via the discriminator's error
        _, g_1, g_2 = gan_model.train_on_batch([z_input, z_labels], [y_gan, z_labels])
        
        # summarize loss on this batch
        print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (i + 1, d_r1, d_r2, d_f, d_f2, g_1, g_2))
        
        # evaluate the model performance every 'epoch'
        if (i + 1) % (bat_per_epo) == 0:
            print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (i + 1, d_r1, d_r2, d_f, d_f2, g_1, g_2))
            print("Synthetic Data Recorded")
            summarize_performance(i, g_model, d_model, latent_dim)

            X_fake = scaler.inverse_transform(X_fake)
            X_fake_temp = X_fake
            labels_fake_temp = labels_fake
            X_fake_temp = X_fake_temp.reshape(n_batch, len(err_train[0]))
            labels_fake_temp = labels_fake_temp.reshape(n_batch, 1)
            generated_fake_data = np.append(X_fake_temp, labels_fake_temp, axis=1)

            np.savetxt('C:/Users/GC/generated_data/generated_fake_data%sth.csv' % (i + 1), generated_fake_data, delimiter=",")
            

In [189]:
latent_dim = 3
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)

In [190]:
discriminator.summary()

Model: "model_33"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_34 (InputLayer)       [(None, 3)]                  0         []                            
                                                                                                  
 dense_77 (Dense)            (None, 16)                   64        ['input_34[0][0]']            
                                                                                                  
 tf.nn.relu_22 (TFOpLambda)  (None, 16)                   0         ['dense_77[0][0]']            
                                                                                                  
 dense_78 (Dense)            (None, 16)                   272       ['tf.nn.relu_22[0][0]']       
                                                                                           

In [191]:
generator.summary()

Model: "model_34"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_35 (InputLayer)       [(None, 3)]                  0         []                            
                                                                                                  
 dense_81 (Dense)            (None, 16)                   64        ['input_35[0][0]']            
                                                                                                  
 tf.math.tanh_22 (TFOpLambd  (None, 16)                   0         ['dense_81[0][0]']            
 a)                                                                                               
                                                                                                  
 dense_82 (Dense)            (None, 16)                   272       ['tf.math.tanh_22[0][0]

In [192]:
gan_model.summary()

Model: "model_35"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_35 (InputLayer)       [(None, 3)]                  0         []                            
                                                                                                  
 dense_81 (Dense)            (None, 16)                   64        ['input_35[0][0]']            
                                                                                                  
 tf.math.tanh_22 (TFOpLambd  (None, 16)                   0         ['dense_81[0][0]']            
 a)                                                                                               
                                                                                                  
 dense_82 (Dense)            (None, 16)                   272       ['tf.math.tanh_22[0][0]

In [ ]:
# load image data
dataset = load_real_samples()
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

(343416, 3) (343416,)
32/32 [==============================] - 0s 1ms/step
>1, dr[0.621,1.763], df[0.747,1.775], g[0.646,1.774]
32/32 [==============================] - 0s 1ms/step
>2, dr[0.616,1.758], df[0.751,1.774], g[0.642,1.770]
32/32 [==============================] - 0s 968us/step
>3, dr[0.612,1.754], df[0.753,1.784], g[0.642,1.784]
32/32 [==============================] - 0s 935us/step
>4, dr[0.609,1.746], df[0.754,1.784], g[0.641,1.782]
32/32 [==============================] - 0s 903us/step
>5, dr[0.607,1.751], df[0.754,1.780], g[0.639,1.777]
32/32 [==============================] - 0s 806us/step
>6, dr[0.604,1.746], df[0.756,1.781], g[0.640,1.775]
32/32 [==============================] - 0s 903us/step
>7, dr[0.601,1.740], df[0.755,1.769], g[0.637,1.775]
32/32 [==============================] - 0s 871us/step
>8, dr[0.597,1.730], df[0.761,1.780], g[0.636,1.775]
32/32 [==============================] - 0s 1ms/step
>9, dr[0.594,1.730], df[0.758,1.777], g[0.637,1.773]
32/32 [=====

32/32 [==============================] - 0s 1000us/step
>76, dr[0.529,1.499], df[0.705,1.882], g[0.710,1.869]
32/32 [==============================] - 0s 903us/step
>77, dr[0.529,1.508], df[0.715,1.888], g[0.696,1.890]
32/32 [==============================] - 0s 806us/step
>78, dr[0.534,1.505], df[0.738,1.895], g[0.673,1.895]
32/32 [==============================] - 0s 871us/step
>79, dr[0.534,1.479], df[0.739,1.887], g[0.676,1.889]
32/32 [==============================] - 0s 903us/step
>80, dr[0.535,1.506], df[0.756,1.897], g[0.663,1.906]
32/32 [==============================] - 0s 903us/step
>81, dr[0.536,1.521], df[0.763,1.914], g[0.645,1.884]
32/32 [==============================] - 0s 968us/step
>82, dr[0.537,1.502], df[0.767,1.903], g[0.634,1.929]
32/32 [==============================] - 0s 839us/step
>83, dr[0.537,1.508], df[0.781,1.919], g[0.629,1.918]
32/32 [==============================] - 0s 903us/step
>84, dr[0.539,1.477], df[0.792,1.911], g[0.617,1.910]
32/32 [===========

32/32 [==============================] - 0s 903us/step
>151, dr[0.656,1.535], df[0.758,1.932], g[0.641,1.938]
32/32 [==============================] - 0s 1ms/step
>152, dr[0.655,1.543], df[0.757,1.957], g[0.642,1.953]
32/32 [==============================] - 0s 968us/step
>153, dr[0.660,1.543], df[0.754,1.935], g[0.643,1.918]
32/32 [==============================] - 0s 903us/step
>154, dr[0.664,1.538], df[0.753,1.944], g[0.645,1.914]
32/32 [==============================] - 0s 871us/step
>155, dr[0.663,1.556], df[0.751,1.941], g[0.648,1.922]
32/32 [==============================] - 0s 968us/step
>156, dr[0.667,1.529], df[0.749,1.932], g[0.646,1.906]
32/32 [==============================] - 0s 1ms/step
>157, dr[0.669,1.546], df[0.747,1.931], g[0.648,1.933]
32/32 [==============================] - 0s 935us/step
>158, dr[0.672,1.538], df[0.745,1.918], g[0.650,1.928]
32/32 [==============================] - 0s 968us/step
>159, dr[0.673,1.546], df[0.745,1.930], g[0.649,1.908]
32/32 [=======

>225, dr[0.734,1.566], df[0.691,1.901], g[0.699,1.892]
32/32 [==============================] - 0s 839us/step
>226, dr[0.732,1.568], df[0.690,1.902], g[0.702,1.909]
32/32 [==============================] - 0s 903us/step
>227, dr[0.734,1.565], df[0.689,1.920], g[0.702,1.903]
32/32 [==============================] - 0s 839us/step
>228, dr[0.735,1.557], df[0.689,1.901], g[0.704,1.912]
32/32 [==============================] - 0s 968us/step
>229, dr[0.734,1.566], df[0.689,1.900], g[0.703,1.910]
32/32 [==============================] - 0s 1ms/step
>230, dr[0.733,1.587], df[0.688,1.896], g[0.704,1.901]
32/32 [==============================] - 0s 1ms/step
>231, dr[0.736,1.562], df[0.687,1.923], g[0.705,1.919]
32/32 [==============================] - 0s 968us/step
>232, dr[0.736,1.573], df[0.687,1.912], g[0.704,1.914]
32/32 [==============================] - 0s 871us/step
>233, dr[0.735,1.585], df[0.686,1.908], g[0.704,1.902]
32/32 [==============================] - 0s 1ms/step
>234, dr[0.734,1

32/32 [==============================] - 0s 1ms/step
>300, dr[0.736,1.550], df[0.673,1.915], g[0.718,1.900]
32/32 [==============================] - 0s 839us/step
>301, dr[0.735,1.567], df[0.673,1.902], g[0.717,1.881]
32/32 [==============================] - 0s 935us/step
>302, dr[0.735,1.553], df[0.675,1.917], g[0.717,1.901]
32/32 [==============================] - 0s 3ms/step
>303, dr[0.734,1.547], df[0.674,1.910], g[0.716,1.904]
32/32 [==============================] - 0s 806us/step
>304, dr[0.733,1.551], df[0.674,1.901], g[0.716,1.885]
32/32 [==============================] - 0s 2ms/step
>305, dr[0.732,1.542], df[0.674,1.896], g[0.716,1.901]
32/32 [==============================] - 0s 3ms/step
>306, dr[0.733,1.557], df[0.676,1.912], g[0.715,1.891]
32/32 [==============================] - 0s 1ms/step
>307, dr[0.732,1.549], df[0.676,1.890], g[0.715,1.904]
32/32 [==============================] - 0s 1ms/step
>308, dr[0.732,1.551], df[0.677,1.906], g[0.714,1.908]
32/32 [===============

...

>Saved: C:/Users/GC/gmodel/generated_plot_1715.png, C:/Users/GC/g_model_1715.h5, C:/Users/GC/d_model_1715.h5 and C:/Users/GC/gan_model_1715.h5
32/32 [==============================] - 0s 1ms/step
>1716, dr[0.642,1.370], df[0.655,1.806], g[0.740,1.799]
32/32 [==============================] - 0s 1ms/step
>1717, dr[0.653,1.362], df[0.650,1.832], g[0.750,1.822]
32/32 [==============================] - 0s 839us/step
>1718, dr[0.647,1.348], df[0.649,1.801], g[0.749,1.794]
32/32 [==============================] - 0s 1ms/step
>1719, dr[0.647,1.331], df[0.654,1.807], g[0.740,1.827]
32/32 [==============================] - 0s 839us/step
>1720, dr[0.638,1.314], df[0.665,1.819], g[0.731,1.826]
32/32 [==============================] - 0s 871us/step
>1721, dr[0.645,1.362], df[0.657,1.812], g[0.741,1.806]
32/32 [==============================] - 0s 903us/step
>1722, dr[0.637,1.353], df[0.650,1.818], g[0.745,1.819]
32/32 [==============================] - 0s 839us/step
>1723, dr[0.642,1.303], df[0.65

32/32 [==============================] - 0s 903us/step
>1789, dr[0.635,1.346], df[0.649,1.818], g[0.754,1.821]
32/32 [==============================] - 0s 1ms/step
>1790, dr[0.640,1.367], df[0.650,1.807], g[0.751,1.808]
32/32 [==============================] - 0s 1ms/step
>1791, dr[0.635,1.369], df[0.647,1.807], g[0.754,1.794]
32/32 [==============================] - 0s 935us/step
>1792, dr[0.632,1.316], df[0.653,1.806], g[0.752,1.815]
32/32 [==============================] - 0s 742us/step
>1793, dr[0.629,1.334], df[0.652,1.826], g[0.748,1.823]
32/32 [==============================] - 0s 1ms/step
>1794, dr[0.634,1.373], df[0.648,1.822], g[0.746,1.812]
32/32 [==============================] - 0s 839us/step
>1795, dr[0.629,1.345], df[0.645,1.793], g[0.757,1.778]
32/32 [==============================] - 0s 839us/step
>1796, dr[0.640,1.373], df[0.641,1.812], g[0.757,1.801]
32/32 [==============================] - 0s 806us/step
>1797, dr[0.641,1.332], df[0.653,1.824], g[0.747,1.817]
32/32 [

32/32 [==============================] - 0s 1ms/step
>1863, dr[0.643,1.345], df[0.642,1.789], g[0.759,1.803]
32/32 [==============================] - 0s 806us/step
>1864, dr[0.648,1.308], df[0.639,1.804], g[0.763,1.795]
32/32 [==============================] - 0s 1ms/step
>1865, dr[0.645,1.367], df[0.647,1.813], g[0.752,1.817]
32/32 [==============================] - 0s 1ms/step
>1866, dr[0.646,1.381], df[0.646,1.806], g[0.755,1.818]
32/32 [==============================] - 0s 903us/step
>1867, dr[0.646,1.406], df[0.651,1.808], g[0.752,1.815]
32/32 [==============================] - 0s 839us/step
>1868, dr[0.643,1.353], df[0.650,1.812], g[0.753,1.812]
32/32 [==============================] - 0s 871us/step
>1869, dr[0.636,1.361], df[0.642,1.794], g[0.761,1.798]
32/32 [==============================] - 0s 839us/step
>1870, dr[0.645,1.409], df[0.646,1.811], g[0.758,1.826]
32/32 [==============================] - 0s 1ms/step
>1871, dr[0.642,1.343], df[0.656,1.813], g[0.755,1.803]
32/32 [==

32/32 [==============================] - 0s 871us/step
>1937, dr[0.639,1.358], df[0.634,1.795], g[0.768,1.787]
32/32 [==============================] - 0s 903us/step
>1938, dr[0.633,1.379], df[0.626,1.802], g[0.773,1.790]
32/32 [==============================] - 0s 1ms/step
>1939, dr[0.639,1.347], df[0.627,1.820], g[0.773,1.796]
32/32 [==============================] - 0s 935us/step
>1940, dr[0.641,1.347], df[0.641,1.799], g[0.764,1.813]
32/32 [==============================] - 0s 871us/step
>1941, dr[0.639,1.326], df[0.645,1.826], g[0.759,1.807]
32/32 [==============================] - 0s 871us/step
>1942, dr[0.630,1.367], df[0.637,1.812], g[0.766,1.798]
32/32 [==============================] - 0s 677us/step
>1943, dr[0.643,1.334], df[0.639,1.806], g[0.761,1.795]
32/32 [==============================] - 0s 871us/step
>1944, dr[0.634,1.359], df[0.639,1.804], g[0.759,1.798]
32/32 [==============================] - 0s 903us/step
>1945, dr[0.639,1.320], df[0.641,1.813], g[0.759,1.823]
32/

32/32 [==============================] - 0s 806us/step
>2011, dr[0.627,1.391], df[0.643,1.806], g[0.760,1.805]
32/32 [==============================] - 0s 839us/step
>2012, dr[0.640,1.366], df[0.641,1.808], g[0.762,1.777]
32/32 [==============================] - 0s 903us/step
>2013, dr[0.628,1.387], df[0.644,1.806], g[0.762,1.792]
32/32 [==============================] - 0s 903us/step
>2014, dr[0.640,1.385], df[0.640,1.807], g[0.764,1.804]
32/32 [==============================] - 0s 871us/step
>2015, dr[0.645,1.344], df[0.639,1.823], g[0.758,1.799]
32/32 [==============================] - 0s 903us/step
>2016, dr[0.644,1.374], df[0.645,1.814], g[0.763,1.817]
32/32 [==============================] - 0s 839us/step
>2017, dr[0.641,1.348], df[0.634,1.814], g[0.764,1.784]
32/32 [==============================] - 0s 871us/step
>2018, dr[0.650,1.327], df[0.628,1.790], g[0.776,1.795]
32/32 [==============================] - 0s 871us/step
>2019, dr[0.639,1.336], df[0.628,1.810], g[0.769,1.791]
3